# 이터페이터와 제너레이터

제너레이터는 return 대신 yield 키워드를 사용하는 함수로 표현되는 것이지만 이터레이터와 어떻게 다른지 확인해 볼 예정입니다. 제너페이터와 이터페이터는 모두 for 루프에서 사용할 수 있습니다. 제너레이터는 특히 모든 데이터를 메모리에 한꺼번에 적재하여 처리하는 대신 적은 메모리로 큰 작업을 수행하는데 유용하게 사용될 수 있습니다.

이전에 파이썬 2 에서 range() 및 xrange() 를 설명할 때 yield가 등장했었습니다. 파이썬 2 에서는 xrange()가 제너페이터였고 파이썬 3에서는 xrange() 가 없어졌고 range() 자체가 제너페이터입니다.

대부분의 경우 제너페이터는 일반 함수와 차이가 거의 없습니다. 하지만 일반 함수와의 차이점은 반복(이터레이션) 프로토콜을 지원하는 객체로 인식된다는 것입니다. yield는 return 처럼 생각하면 되지만 실제로 해당 값으로 해당 함수를 끝내는 것이 아니라 다음 제너레이터가 호출되었을 때는 이전에 yield 했던 그 다음에 계속 진행된다는 것이 일반 함수와 다른 점입니다. 
이런 것은 *상태 보관 (state suspension)* 이라는 기능을 이용하여 제너레이터의 상태를 보관하고 있다가 다음번에 호출되었을 때 계속 진행할 수 있게 되는 것입니다.

다음과 같은 예로 시작해 봅시다.

In [1]:
# 주어진 값의 3 제곱을 구하는 제터레이터를 작성해 보았습니다
def gencubes(n):
    for num in range(n):
        yield num**3

In [2]:
for x in gencubes(10):
    print(x)

0
1
8
27
64
125
216
343
512
729


여기서는 yield 대신 return을 사용해도 동일한 결과가 나올 것입니다.

제너레이터는 매우 큰 결과를 계속하여 작업할 때 모든 결과를 미리 만들어 놓지 않고 일을 계속해서 수행하는데 이용하면 좋습니다.

다음의 [피보나치 수열](https://ko.wikipedia.org/wiki/%ED%94%BC%EB%B3%B4%EB%82%98%EC%B9%98_%EC%88%98)에 관련된 예제를 확인해 보겠습니다.

In [3]:
def genfibon(n):
    '''
    n 까지의 피보나치 수열을 구하는 제너레이터
    '''
    a = 1
    b = 1
    for i in range(n):
        yield a
        a,b = b,a+b

In [4]:
for num in genfibon(10):
    print(num)

1
1
2
3
5
8
13
21
34
55


제너레이터가 아니라 일반 함수로 표현해 보겠습니다.

In [5]:
def fibon(n):
    a = 1
    b = 1
    output = []
    
    for i in range(n):
        output.append(a)
        a,b = b,a+b
        
    return output

In [6]:
fibon(10)

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

위에서와 같이 10 까지의 피보나치 수열을 구한다면 제너레이터나 일반 함수나 큰 차이가 나지 않을 것입니다. 하지만 만약 n 이 큰 수 (예를 들어 1,000,000 과 같은) 라고 가정해 보겠습니다. 제너레이터는 내부적으로 메모리 상에 결과를 계속 가지고 있지 않지만 일반 함수는 해당 개수 만큼 리스트에 결과 값을 담고 있을 것입니다. 동일한 결과를 처리하는데 있어 함수 보다는 제너레이터를 이용하면 메모리를 크게 절약할 수 있음을 알 수 있습니다.

## next() 와 iter() 빌트인 함수

제너레이터를 더 확실히 알려고 하려면 next() 및 iter() 빌트인 함수를 알아야 합니다.

다음의 예를 확인해 보겠습니다.

In [7]:
def simple_gen():
    for x in [0, 1, 2]:
        yield x

In [8]:
# simple_gen 이라는 제너레이터를 g 변수에 assign 하였습니다.
g = simple_gen()

In [9]:
print(next(g))

0


In [10]:
print(next(g))

1


In [11]:
print(next(g))

2


In [12]:
print(next(g))

StopIteration: 

next() 함수를 이용하여 제너페이터를 계속하여 결과를 가져오다 마지막이 되면 StopIteration 오류가 발생합니다. 이 오류는 제너페이터가 종료되었음을 의미합니다. 그러면 for 루프에서는 어떻게 이런 오류가 발생하지 않는 것일까요? 그 이유는 이 오류가 발생하면 자동으로 for 루프를 빠져 나오기 때문입니다.

다음에는 iter() 를 확인해 보겠습니다.

우선 처음에 확인해 보았던 문자열을 확인해 보겠습니다.

In [13]:
s = 'hello'

#Iterate over string
for let in s:
    print(let)

h
e
l
l
o


for 루프에서 위와 같은 문자열을 이용할 수는 있지만 문자열 자체가 이터레이터가 아니라 next() 함수를 이용할 수는 없습니다. 다른 말로 말하면 이터레이터는 next() 빌트인 함수를 호출하여 그 다음 값을 가져올 수 있는 객체를 의미합니다. 문자열 자체는 이터레이터가 아니므로 문자열로 부터 이터레이터를 구해야 하는데 그 때 이용할 수 있는 함수가 iter() 입니다.

우선 다음과 같이 문자열에 대하여 next()를 호출해 보면,

In [14]:
next(s)

TypeError: 'str' object is not an iterator

오류가 발생합니다. 하지만,

In [15]:
s_iter = iter(s)

In [16]:
next(s_iter)

'h'

In [17]:
next(s_iter)

'e'

iterm() 로 얻은 s_iter 이라는 문자열 이터레이터는 next()  를 호출하여 다음 값을 구할 수 있었습니다.